<a href="https://colab.research.google.com/github/jmem-ec/KRRCourse/blob/main/Practica-9/Practica_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Módulo Representación del Conocimiento - Práctica-9

Ejemplo **código Python** para la práctica 9 del módulo Representacion del Conocimiento en la Maestría de Ciencia de Datos

## Instalacion de librerias

In [2]:
!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON, XML
from pprint import pprint
import time
import os

In [3]:
def loadingData(endpoint_url, file, format="ttl"):
    #Informacion para la conexion extraida de
    #https://graphdb.ontotext.com/documentation/free/quick-start-guide.html#load-data-through-sparql-or-rdf4j-api

    print("Uploading file: " + file)
    if format=="trig":
        curl_command = "curl '" + endpoint_url + "/statements' -X POST -H \"Content-Type:application/x-trig\" -T '" + file + "'"
    else:
        curl_command = "curl '" + endpoint_url + "/statements' -X POST -H \"Content-Type:application/x-turtle\" -T '" + file + "'"

    #Other formats: https://librdf.org/raptor/api/raptor-formats-types-by-parser.html
    #print(curl_command)

    status = os.system(curl_command)
    #print(status)

In [4]:
def queryGraphDBRepo(endpoint_url, query, attempts=3):


    try:

        sparql_web = SPARQLWrapper(endpoint_url)
        # Default is XML:
        # https://sparqlwrapper.readthedocs.io/en/latest/SPARQLWrapper.Wrapper.html
        sparql_web.setReturnFormat(JSON)

        sparql_web.setQuery(query)

        results = sparql_web.query().convert()

        #Raw results in json format
        #print("RAW RESULTS IN JSON FORMAT:")
        #pprint(results)

        print("\tRetrieved tuples: " + str(len(results["results"]["bindings"])))

        #Processed results
        #print("Processed results in CSV format:")
        for result in results["results"]["bindings"]:
            row =""
            for out_var in results["head"]["vars"]:
                #print(out_var)
                #print(result[out_var]['value'])
                row = row + "\"" + result[out_var]['value'] + "\","

            print(row)


    except Exception as e:
        print(e)

        print("Query '%s' failed. Attempts: %s" % (query, str(attempts)))
        time.sleep(1) #to avoid limit of calls, sleep 1s
        attempts-=1
        if attempts>0:
            return queryGraphDBRepo(endpoint_url, query, attempts)
        else:
            return None


###Ejemplo de carga de datos using CURL

In [5]:
localhost = "http://127.0.0.1:7200"

graphdb_endpoint = localhost + "/repositories/ejemploDB"
format = "ttl"

##Load ontology
path_to_onto_file = "https://github.com/jmem-ec/KRRCourse/blob/d32490b06421111680c143c78f8ba557476b33ba/Practica-9/AcademicKG.ttl"
loadingData(graphdb_endpoint, path_to_onto_file, format)

##Load data
path_to_data_file = "https://github.com/jmem-ec/KRRCourse/blob/d32490b06421111680c143c78f8ba557476b33ba/Practica-9/DataAcademicKG.ttl"
loadingData(graphdb_endpoint, path_to_data_file, format)


Uploading file: https://github.com/jmem-ec/KRRCourse/blob/d32490b06421111680c143c78f8ba557476b33ba/Practica-9/AcademicKG.ttl
Uploading file: https://github.com/jmem-ec/KRRCourse/blob/d32490b06421111680c143c78f8ba557476b33ba/Practica-9/DataAcademicKG.ttl


##Ejemplo de consulta

In [ ]:
print("\nQuerying GraphDB Endpoint:")

query = """
PREFIX : <http://example.org/university/ucuenca/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?profesor ?nombre
WHERE {
  ?profesor rdf:type :Docente .
  ?profesor :nombre ?nombre .

  MINUS {
    ?profesor :dicta ?curso .
  }
}
"""

queryGraphDBRepo(graphdb_endpoint, query)
